In [149]:
%load_ext autoreload
%autoreload 2
import os
import random
import sqlite3 
import numpy as np
import json
import math
from tqdm.notebook import tqdm
from tqdm import tqdm
tqdm.pandas()
import calendar
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import Advan_operator as ad_op  
import helper

pd.set_option('display.max_columns', None)

import importlib
import helper
importlib.reload(helper)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'helper' from 'd:\\OneDrive_Emory\\OneDrive - Emory\\Research_doc\\hourly_population\\github\\hourly_population\\helper.py'>

In [114]:

# data_path = r'D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\hourly_results'
# save_path = r'D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\hourly_results\maps'

data_path = r'D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\hourly_results\removed_negative'
save_path = r'D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\hourly_results\maps_removed_negative_all'

os.makedirs(save_path, exist_ok=True)
os.makedirs(os.path.join(save_path, 'place_plots'), exist_ok=True)

years = [2022]
months = list(range(1, 13))

landscan_daytime_fname =   r"D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\data\Landscan_daytime_2021_CBG.csv"
landscan_nighttime_fname = r"D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\data\Landscan_nighttime_2021_CBG.csv"

# hourly_popu_fname = fr"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_dell_add_stop_factor\CBG_population_hourly_{year}{month:02}.csv"
ACS_file = r"D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\data\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
CBG_place_fname = r'D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\data\CBG_place.gpkg'

CBG_2019_fname = r"D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\data\blockgroups2019.zip"


landscan_day_df = pd.read_csv(landscan_daytime_fname, dtype={"GEOID":str}, usecols=['GEOID', 'SUM']).rename(columns={"SUM": "landscan_day", "GEOID":"CBG"}).set_index("CBG").astype(int)
landscan_night_df = pd.read_csv(landscan_nighttime_fname, dtype={"GEOID":str}, usecols=['GEOID', 'SUM']).rename(columns={"SUM": "landscan_night", "GEOID":"CBG"}).set_index("CBG").astype(int)

ACS_df = pd.read_csv(ACS_file, dtype={'fips':str}).iloc[:, :2].rename(columns={"fips": "CBG"}).set_index("CBG").astype(int)
ACS_df = ACS_df.merge(landscan_day_df, left_index=True, right_index=True).merge(landscan_night_df, left_index=True, right_index=True)
ACS_df

,totalpopulation,landscan_day,landscan_night
CBG,,,
010010201001,730,392,580
010010201002,1263,436,1194
010010202001,835,1622,904
010010202002,1124,2566,1116
010010203001,2774,974,2254
...,...,...,...
560459511001,1450,951,1408
560459511002,1844,956,1798
560459513001,1314,803,1041


In [118]:
import geopandas as gpd
state_fname = r'https://github.com/gladcolor/spatial_data/raw/refs/heads/master/cb_2019_us_state_20m.zip'
state_gdf = gpd.read_file(state_fname)
state_gdf.rename(columns={'NAME':'STATE_NAME'}, inplace=True)


county_fname = r'https://github.com/gladcolor/spatial_data/raw/refs/heads/master/cb_2019_us_county_20m.zip'
county_gdf = gpd.read_file(county_fname)
county_gdf['county_FIPS'] = county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.rename(columns={'NAME':'COUNTY_NAME'}, inplace=True)

county_gdf = county_gdf.merge(state_gdf[['STATEFP','STATE_NAME', 'STUSPS']], on='STATEFP', how='left', validate='m:1', indicator=True)
county_gdf.sort_values(['STATEFP','COUNTYFP'], inplace=True)
county_gdf = county_gdf[['STATEFP', 'STUSPS', 'STATE_NAME',  'county_FIPS', 'COUNTY_NAME' ]].set_index('county_FIPS')
# validate='m:1' asserts each county matches at most one state (many counties to one state); if the source breaks that, merge raises an error instead of silently duplicating rows. Huan: doesn't seem to happen here.
# indicator=True adds a _merge column showing whether the state match came from both dataframes or not, handy for spotting counties that failed to match.
county_gdf

,STATEFP,STUSPS,STATE_NAME,COUNTY_NAME
county_FIPS,,,,
01001,01,AL,Alabama,Autauga
01003,01,AL,Alabama,Baldwin
01005,01,AL,Alabama,Barbour
01007,01,AL,Alabama,Bibb
01009,01,AL,Alabama,Blount
...,...,...,...,...
72145,72,PR,Puerto Rico,Vega Baja
72147,72,PR,Puerto Rico,Vieques
72149,72,PR,Puerto Rico,Villalba


In [87]:
month = 7
year = 2022
CBG = '120860099031'

hourly_popu_fname = fr"D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\hourly_results\CBG_population_hourly_{year}{month:02}.csv"
print("Loading:", hourly_popu_fname)
df = pd.read_csv(hourly_popu_fname, dtype={'CBG':str}).set_index('CBG').astype(int) 
df

Loading: D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\hourly_results\CBG_population_hourly_202207.csv


,2022-07-01 00:00:00,2022-07-01 01:00:00,2022-07-01 02:00:00,2022-07-01 03:00:00,2022-07-01 04:00:00,2022-07-01 05:00:00,2022-07-01 06:00:00,2022-07-01 07:00:00,2022-07-01 08:00:00,2022-07-01 09:00:00,...,2022-07-31 14:00:00,2022-07-31 15:00:00,2022-07-31 16:00:00,2022-07-31 17:00:00,2022-07-31 18:00:00,2022-07-31 19:00:00,2022-07-31 20:00:00,2022-07-31 21:00:00,2022-07-31 22:00:00,2022-07-31 23:00:00
CBG,,,,,,,,,,,,,,,,,,,,,
010010201001,538,596,646,668,706,647,669,929,427,399,...,389,-32,-21,93,166,153,219,130,347,453
010010201002,1305,1300,1283,1325,1220,1106,861,547,470,512,...,740,678,859,1269,1505,1724,1779,1569,1556,1481
010010202001,715,689,790,768,759,695,568,443,248,120,...,198,-89,-123,291,111,82,134,151,526,532
010010202002,1009,986,1102,1086,1052,1229,1155,2022,1946,2610,...,1376,1106,1707,2284,1587,1468,2019,1010,1051,1227
010010203001,2696,2738,2753,2716,2643,2505,2199,1879,2059,2272,...,2684,2768,3177,3533,3939,3813,3399,3204,3140,3052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560459511001,1301,1342,1361,1385,1478,1449,1505,1743,1678,1543,...,1205,1028,1174,1217,1389,1160,840,1002,1062,1135
560459511002,1826,1966,1831,1828,1817,1907,1887,1926,1982,1648,...,1168,1325,1341,1004,942,1384,1400,1484,1847,1741
560459513001,1178,1252,1324,1340,1264,1292,1404,1244,1059,1060,...,875,1011,665,1038,1324,1078,1053,1400,1238,1132


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

import helper

import pandas as pd
import numpy as np
from scipy.signal import find_peaks

import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from matplotlib.patches import Patch


from scipy.signal import find_peaks, peak_widths
import numpy as np
import pandas as pd
import gc
 
save_dir = r'D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\event_detection'
os.makedirs(save_dir, exist_ok=True)
 

def event_detection_CBG(df, CBG, year, month, county_gdf, save_dir):
    # ---- Legend patches for background spans ----
    s = df.loc[CBG]

    weekend_patch = Patch(
        facecolor="lightgreen",
        alpha=0.25,
        label="Weekend"
    )

    night_patch = Patch(
        facecolor="lightgrey",
        alpha=0.25,
        label="Nighttime (7 PM–7 AM)"
    )

    # CBG = '120860099031'
    # CBG = '360610143001' # Central Park
    # CBG = '280539503003' #  World Catfish Festival,April 
    # CBG = '540259505003'

    fig, ax = plt.subplots(figsize=(20, 6))
 
    helper.plot_hourly_with_context(
        s,
        ax=ax,
        title=f"Hourly Population for CBG {CBG} in {year}-{month:02}"
    )

    baseline = np.quantile(s.values, 0.75)
    peaks = helper.detect_hourly_peaks(
        s,
        baseline=baseline,
        min_prominence_ratio=5,
        min_distance_hours=12,
        min_height_quantile=0.90
    )

    handles = [weekend_patch, night_patch]


    peak_handle = plt.Line2D(
        [], [], 
        color="red", 
        marker="o", 
        linestyle="None",
        markersize=6,
        label="Detected peaks"
        )

    handles.append(peak_handle)

    # if find peaks, plot them, save them and the plot
    if len(peaks) > 0:
        county_info = county_gdf.loc[CBG[:5]]
        county_name = county_info['COUNTY_NAME']
        state_name = county_info['STATE_NAME']

        # draw peaks
        ax.scatter(
            peaks["time"],
            peaks["value"],
            color="red",
            s=40,
            zorder=5,
            label="Detected peaks"
        )

        ax.legend(
            handles=handles,
            loc="upper right",
            frameon=True
        )
    
        plot_fname = os.path.join(save_dir,   f'{state_name}_{county_name}_CBG_{CBG}_{year}{month:02}_peaks.png')
        plt.savefig(plot_fname, dpi=300, bbox_inches='tight')
        peaks_fname = os.path.join(save_dir, f'{state_name}_{county_name}_CBG_{CBG}_{year}{month:02}_peaks.csv')
        peaks.to_csv(peaks_fname, index=False)
        
    plt.close(fig)
    gc.collect()

        # print(peaks)
 

for month in months:
    hourly_popu_fname = fr"D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\hourly_results\removed_negative\CBG_population_hourly_{year}{month:02}.csv"
    print("Loading:", hourly_popu_fname)
    df = pd.read_csv(hourly_popu_fname, dtype={'CBG':str})
    df['CBG'] = df['CBG'].str.zfill(12)
    df = df.set_index('CBG').astype(int)

    for CBG in tqdm(df.index.tolist()):
        event_detection_CBG(df, CBG, year, month, county_gdf, save_dir)
# plt.tight_layout()
# plt.show()

Loading: D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\hourly_results\removed_negative\CBG_population_hourly_202201.csv


  0%|          | 21/216725 [00:01<2:43:05, 22.14it/s]C:\tmp\ipykernel_42808\3761513210.py:47: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(20, 6))
 21%|██        | 45131/216725 [11:19:19<109:44:52,  2.30s/it] 